In [2]:
import streamlit as st
import streamlit.components.v1 as components
import csv
import string
import os
import pickle
import pandas as pd
from datetime import datetime as dt
from base64 import b64encode
#import proximitypyhash as pph
#import pygeohash as gh
from streamlit_folium import st_folium
from streamlit_folium import folium_static
import folium
import numpy as np

from agile.filtering import query_location, query_date, query_adid, query_node
from agile.mapping import data_map 
from agile.locations import locations_of_interest
from agile.people import colocation
from agile.prediction import double_cluster, get_top_N_clusters
from agile.utils.tag import find_all_nearby_nodes
from agile.utils.geocode import reverse_geocode
from agile.utils.files import find, random_line, save, random_name, generate_aliases
from agile.utils.dataframes import modify_and_sort_columns, clean_and_verify_columns
from agile.profile import Profile
from agile.samsreport import Report
from agile.centrality import compute_top_centrality
from agile.overview import adid_value_counts

from streamlit_option_menu import option_menu
import pygeohash as gh

In [12]:
df = pd.read_csv('data/too_many_adids.csv')

In [13]:
df['datetime'] = pd.to_datetime(df['datetime'],errors='coerce')

In [14]:
df = modify_and_sort_columns(df)

In [15]:
aliases = generate_aliases(df)
if len(aliases) < len(df['advertiser_id'].unique()):
    print("WARNING: Due to the amount of ADIDs in your data, not every ADID was assigned an alias")

starting file reading
ending file reading
1368896 1500000


In [6]:
len(df['advertiser_id'].unique())

10605

In [7]:
first = []
last = []
name_list = []

with open(find('../names/first.txt')) as F, open(find('../names/last.txt')) as L:
    for line_f in F:
        first.append(line_f.strip())
 
    for line_l in L:
        last.append(line_l.strip())
        
    for f in first:
        for l in last:        
            name_list.append(l + '-' + f)
            
    

In [8]:
import random
adid_dict = {}
for id in df['advertiser_id'].unique():
    adid_dict[id] = name_list.pop(random.randrange(len(name_list)))



## ADID Alias Error Check

In [14]:
import random
import string
from datetime import datetime, timedelta

In [12]:
def generate_random_string(length):
    return ''.join(random.choices(string.ascii_letters + string.digits, k=length))

def generate_unique_combinations(total_combinations):
    generated_combinations = set()
    while len(generated_combinations) < total_combinations:
        generated_combinations.add(generate_random_string(10))  # Change the length of string if needed
    return list(generated_combinations)

def generate_datetime_objects(total_objects):
    start_date = datetime(2000, 1, 1)
    end_date = datetime(2023, 12, 31)
    delta = end_date - start_date

    datetime_objects = [start_date + timedelta(days=random.randint(0, delta.days)) for _ in range(total_objects)]
    return datetime_objects

def generate_random_numbers(total_numbers):
    random_numbers = [random.random() for _ in range(total_numbers)]
    return random_numbers

In [11]:
total_combinations = 1500000
combinations = generate_unique_combinations(total_combinations)
print(f"Total unique combinations generated: {len(combinations)}")

Total unique combinations generated: 1500000


In [16]:
total_datetime_objects = 1500000
total_random_numbers = 1500000

datetime_objects = generate_datetime_objects(total_datetime_objects)
latitudes = generate_random_numbers(total_random_numbers)
longitudes = generate_random_numbers(total_random_numbers)

In [25]:
testing_df = pd.DataFrame(columns=['datetime','advertiser_id','latitude','longitude','geohash'])
testing_df['datetime'] = datetime_objects
testing_df['advertiser_id'] = combinations
testing_df['latitude'] = latitudes
testing_df['longitude'] = longitudes
testing_df['geohash'] = ['c25r0uhu56'] * len(testing_df)

In [26]:
testing_df.to_csv('data/too_many_adids.csv')